## Here we will have the sectors index computation:
### we will start each sector on the date the last stock has ipoed, or 10 years, deciding which is older. after new stock we rebalance but keep the value the same.
- data/numbers/market/SX/sectors:
    - sector.json: dict(stocks:list(ticker, perc_val), last_rebalance:datetime, ) 
    - sector.parquet: table(columns=[date, open, close, high, low, volume])
    - info.json: sec_dict(sector:list([stock, subsector]))

1. We rebalance if new stock comes
2. We rebalance each year??


In [4]:
root_path= "/Users/a2/code/fin/trade/data/price/market"
SX = "AT"


In [11]:
import os, json

p = "/Users/a2/code/fin/trade/data/fundamentals/company/info"
files = os.listdir(p)

with open("/Users/a2/code/fin/trade/static/tickers/AT_shitticker.json") as f:
    sh = json.load(f)

infos = []
for n, f in enumerate(files):
    if f.replace("json", "AT") in sh:
        continue
    with open(os.path.join(p, f)) as fp:
        t = json.load(fp)

    try:
        t["sector"]
    except:
        print(f)
    infos.append(t)





In [3]:
dd:dict[str, list[str]] = {}

for i in infos:

    if i["sector"] not in dd.keys():
        dd[i['sector']] = []
    if i['subsector'] not in dd[i['sector']]:
        dd[i['sector']].append(i['subsector'])

In [4]:
d_num = {}
for i in infos:
    if i["sector"] not in d_num.keys():
        d_num[i["sector"]] = 1  
    else:
        d_num[i["sector"]] +=1

In [5]:
d_num

{'Health Care': 3,
 'Retail': 7,
 'Industrial Goods & Services': 19,
 'Banks': 8,
 'Real Estate': 13,
 'Basic Resources': 17,
 'Technology': 11,
 'Personal Care, Drug & Grocery Stores': 2,
 'Telecommunications': 2,
 'Consumer Products & Services': 8,
 'Media': 2,
 'Construction & Materials': 13,
 'Utilities': 6,
 'Energy': 4,
 'Travel & Leisure': 6,
 'Insurance': 1,
 'Food Beverage & Tobacco': 9,
 'Financial Services': 5}

In [14]:
# wir bauen einen index für sektor.
sec_dict = {} # 2d array: ticker, subsector
for k in d_num.keys():
    sec_dict[k.strip()] = []

# name-> ticker
with open("/Users/a2/code/fin/trade/static/tickers/AT_name_ticker_en.json") as f:
    nt = json.load(f)



In [ ]:
for i in infos:
    try:
        sec_dict[i['sector'].strip()].append([nt[i["name"]], i["subsector"].strip()])
    except:
        print(i["name"])

In [16]:
sec_dict

{'Health Care': [['IATR', 'Health Care Services'],
  ['LAVI', 'Pharmaceuticals'],
  ['MEDIC', 'Medical Equipment']],
 'Retail': [['LANAC', 'Apparel Retailers'],
  ['BELA', 'Diversified Retailers'],
  ['ASCO', 'Specialty Retailers'],
  ['MODA', 'Home Improvement Retailers'],
  ['FOYRK', 'Diversified Retailers'],
  ['MOTO', 'Specialty Retailers'],
  ['NAKAS', 'Specialty Retailers']],
 'Industrial Goods & Services': [['FLEXO', 'Containers & Packaging'],
  ['EUROC', 'Back Office Support HR & Consulting'],
  ['DAIOS', 'Plastics'],
  ['HAIDE', 'Forms & Bulk Printing Services'],
  ['FRIGO', 'Machinery: Industrial'],
  ['PPA', 'Transportation Services'],
  ['BIOSK', 'Machinery: Industrial'],
  ['VOSYS', 'Industrial Suppliers'],
  ['VIS', 'Containers & Packaging'],
  ['ELTON', 'Industrial Suppliers'],
  ['CENER', 'Electrical Components'],
  ['INTET', 'Industrial Suppliers'],
  ['PETRO', 'Commercial Vehicles & Parts'],
  ['PLAKR', 'Plastics'],
  ['AIA', 'Transportation Services'],
  ['PLAT', 'Pl

In [18]:
with open(os.path.join(root_path, SX, "sectors", "info.json"), "w") as f:
    json.dump(sec_dict, f)

### until here was preprocess

In [8]:
root_path= "/Users/a2/code/fin/trade/tradx/data/price/market"
SX = "AT"

In [9]:
import os, json
with open(os.path.join(root_path, SX, "sectors", "info.json"), "r") as f:
    dd:dict = json.load(f)


In [10]:
def normalize_sector_name(sector_name:str)->str:
    return sector_name.lower().replace(" ", "").replace("/", "-")

In [11]:
import datetime

for sector in dd.keys():
    sector_name = normalize_sector_name(sector_name=sector)
    sector_json = {}
    sector_json["stocks"] = [[x[0], 0] for x in dd[sector]]
    with open(os.path.join(root_path, SX, "sectors", sector_name+".json"), "w") as f:
        json.dump(sector_json,f)

In [23]:
with open("/Users/a2/code/fin/trade/tradx/static/tickers/AT_shitticker.json") as f:
    sh = json.load(f)

In [27]:
import yfinance as yf, os, json


non_equity = []
for file in os.listdir(os.path.join(root_path, SX, "sectors")):
    if not file.endswith(".json"):
        continue
    if file == "info.json":
        continue
    with open(os.path.join(root_path, SX, "sectors", file), "r") as f:
        sector_obj = json.load(f)
    mcaps = []
    for sx in sector_obj['stocks']:
        tt = sx[0]+".AT"
        if tt not in sh:
            ticker = yf.Ticker(tt)
            info = ticker.info
            if info['quoteType'] != "EQUITY":
                non_equity.append(tt)
                mcaps.append(0)
                continue

            try:
                mcaps.append(info['marketCap'])
            except:
                print(tt)
                raise RuntimeError
    if len(mcaps) == 0:
        print(file)
        continue
    full_sector_mcap = sum(mcaps)
    # we define the weights of the stocks in the sector.
    for i, sx in enumerate(sector_obj['stocks']):
        sx[1] = mcaps[i]/full_sector_mcap
    with open(os.path.join(root_path, SX, "sectors", file), "w") as f:
        json.dump(sector_obj, f)



In [16]:
sh.append("MLS.AT")
yf.Ticker("HTO.AT").info

{'address1': '99 Kifissias Avenue',
 'address2': 'Maroussi',
 'city': 'Athens',
 'zip': '151 24',
 'country': 'Greece',
 'website': 'https://www.cosmote.gr',
 'industry': 'Telecom Services',
 'industryKey': 'telecom-services',
 'industryDisp': 'Telecom Services',
 'sector': 'Communication Services',
 'sectorKey': 'communication-services',
 'sectorDisp': 'Communication Services',
 'longBusinessSummary': 'Hellenic Telecommunications Organization S.A. engages in the provision of telecommunications and related services to businesses and individuals primarily in Greece and Romania. It operates through OTE, COSMOTE Group \x96 Greece, and Telekom Romania Mobile segments. The company offers fixed and mobile telephony, broadband, information and communication technology, and pay television services; and international carrier services. It also provides mobile and satellite telecommunication, electronic money, e-commerce, financing, consultancy and security, real estate, insurance brokerage, trai

In [35]:
non_equity

['MLS.AT', 'FIER.AT', 'DUR.AT']

In [29]:
sh

['G090312C1.AT', 'CMPB1.AT', 'CPLPB1.AT', 'SBB1.AT', 'TENERGFB2.AT', 'AETF.AT']

In [30]:
with open("/Users/a2/code/fin/trade/tradx/static/tickers/AT_non_equity.json", "w") as f:
    json.dump(non_equity, f)


In [37]:
for _ in non_equity:
    with open(("/Users/a2/code/fin/trade/tradx/data/fundamentals/company/info/AT/"+ _).replace(".AT", "")+ ".json", "r") as f:
        t = json.load(f)
    print(t["webpage"])


None
None
None


In [2]:
import pandas as pd

df = pd.read_parquet("/Users/a2/code/fin/trade/tradx/data/price/company/AT/daily/ALPHA.AT.parquet")

In [10]:
df.iloc[-1, 3]

np.float64(1.5800000429153442)

In [11]:
df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-11-11 00:00:00+02:00,1.5100,1.578,1.505,1.5735,14049136,0.0,0.0
2024-11-12 00:00:00+02:00,1.5660,1.623,1.562,1.6195,16807315,0.0,0.0
2024-11-13 00:00:00+02:00,1.5950,1.627,1.583,1.6045,16071129,0.0,0.0
2024-11-14 00:00:00+02:00,1.6045,1.618,1.588,1.6065,7440826,0.0,0.0
2024-11-15 00:00:00+02:00,1.5835,1.590,1.561,1.5800,2098985,0.0,0.0


In [14]:
import datetime
df.loc[datetime.date(2024, 12, 4), "Close"]

KeyError: datetime.date(2024, 12, 4)

In [26]:
datetime.date(1987, 1, 2) in df.index

False

In [27]:
df.index[0]

Timestamp('1987-01-02 00:00:00+0200', tz='Europe/Athens')

In [41]:
from pandas.core.indexes.datetimes import Timestamp


pd.read_parquet("/Users/a2/code/fin/trade/tradx/data/price/company/AT/daily/IATR.AT.parquet").loc[Timestamp('2014-12-06 00:00:00+0200', tz='Europe/Athens'), "Close"]

KeyError: Timestamp('2014-12-06 00:00:00+0200', tz='Europe/Athens')

In [36]:
pd.read_parquet("/Users/a2/code/fin/trade/tradx/data/price/market/AT/sectors/energy.parquet").head(50)

,date,value
0,2014-12-04 00:00:00+02:00,3.052420
1,2014-12-05 00:00:00+02:00,3.302971
2,2014-12-06 00:00:00+02:00,0.000000
3,2014-12-07 00:00:00+02:00,0.000000
4,2014-12-08 00:00:00+02:00,3.216247
5,2014-12-09 00:00:00+02:00,2.898904
6,2014-12-10 00:00:00+02:00,2.814071
7,2014-12-11 00:00:00+02:00,2.774475
8,2014-12-12 00:00:00+02:00,2.687875
9,2014-12-13 00:00:00+02:00,0.000000
